In [ ]:
import numpy as np
import tensorflow as tensorflow
import time
import math

In [ ]:
voxel_train = np.load('voxel_train.npy')
gt_train = np.load('gt_train.npy')

In [ ]:
#Define Loss functions

arr=np.logspace(math.log10(10.0), math.log10(2000.), num=1, endpoint=True, base=10.0) # num = number of output

arr=np.tile(arr, (2000, 1)) #(x,y) x = batch size

arr_tf=tf.constant(arr.astype('float32'), dtype=tf.float32)

#Implementation of the Wasserstein Distance
def wasserstein_distance(y_actual,y_pred):

    #np.abs(np.cumsum(gt_distributions[40,:]-dist_array[40,:])      
    abs_cdf_difference=tf.math.abs(tf.math.cumsum(y_actual-y_pred,axis=1))

    return tf.reduce_mean(0.5*tf.reduce_sum(tf.math.multiply(-arr_tf[:,:-1]+arr_tf[:,1:],abs_cdf_difference[:,:-1]+abs_cdf_difference[:,1:]),axis=1))

#Combination loss function used in MIML
def MSE_wasserstein_combo(y_actual,y_pred):

    wass_loss=wasserstein_distance(y_actual,y_pred)
    MSE= tf.math.reduce_mean(tf.reduce_mean(tf.math.squared_difference(y_pred, y_actual),axis=1))

    return wass_loss+100000.*MSE

In [ ]:
#Define the network structure
inputs = tf.keras.Input(shape=(6,)) #number of inputs
x = tf.keras.layers.Dense(256, activation=tf.nn.leaky_relu, kernel_initializer='he_uniform',bias_initializer=tf.keras.initializers.Constant(0.01))(inputs)
x = tf.keras.layers.Dense(256, activation=tf.nn.leaky_relu, kernel_initializer='he_uniform',bias_initializer=tf.keras.initializers.Constant(0.01))(x)
x = tf.keras.layers.Dense(256, activation=tf.nn.leaky_relu, kernel_initializer='he_uniform',bias_initializer=tf.keras.initializers.Constant(0.01))(x)
x = tf.keras.layers.Dense(256, activation=tf.nn.leaky_relu, kernel_initializer='he_uniform',bias_initializer=tf.keras.initializers.Constant(0.01))(x)
x = tf.keras.layers.Dense(256, activation=tf.nn.leaky_relu, kernel_initializer='he_uniform',bias_initializer=tf.keras.initializers.Constant(0.01))(x)
x = tf.keras.layers.Dense(256, activation=tf.nn.leaky_relu, kernel_initializer='he_uniform',bias_initializer=tf.keras.initializers.Constant(0.01))(x)
outputs = tf.keras.layers.Dense(1, activation=tf.keras.activations.linear, kernel_initializer='he_uniform',bias_initializer=tf.keras.initializers.Constant(0.01))(x) # no. of outputs
#outputs=tf.keras.layers.Dense(1, activation=tf.keras.activations.relu, kernel_initializer='normal',bias_initializer=tf.keras.initializers.Constant(0.1))(x)
model = tf.keras.Model(inputs=inputs, outputs=outputs)

In [ ]:
#Define optimizer and train the network

#To save the model for each epoch, uncomment the checkpoint_callback below
# checkpoint_callback=tf.keras.callbacks.ModelCheckpoint('weights.{epoch:02d}-{val_loss:.2f}.hdf5', monitor='val_loss', verbose=0, save_best_only=False, save_weights_only=True mode='auto', save_freq='epoch')

# opt = tf.keras.optimizers.Adam(learning_rate=0.001)

# model.compile(optimizer=opt, loss=MSE_wasserstein_combo, metrics=['mse',wasserstein_distance])

# start=time.time()

# model.fit(signals_train, distributions_train,epochs=10, batch_size=10, validation_data=(signals_valid,distributions_valid))  # starts training

# Original parameters
# model.fit(signals_train, distributions_train,epochs=30, batch_size=2000, validation_data=(signals_valid,distributions_valid),callbacks=[checkpoint_callback])  # starts training


# end=time.time()

# print('Time Elapsed:%i seconds'%(end-start))